In [1]:
import pandas as pd
import numpy as np
import datetime
import plotly
import plotly.graph_objs as go

In [2]:
locations = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/locations.csv", usecols=['location'])

In [3]:
locations_labels = locations['location']
locations_values = [nome.replace(' ', '%20') for nome in locations['location']]

In [4]:
locations

,location
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
...,...
190,Venezuela
191,Vietnam
192,Wales
193,Zambia


In [5]:
from fbprophet import Prophet

In [6]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

In [7]:
app = dash.Dash(__name__, external_stylesheets = [dbc.themes.CYBORG],
                meta_tags=[{'name': 'viewport',
                             'content': 'width=device-width initial-scale=1.0'
                           } 
                          ]
               )
app.title = 'Número de pessoas vacinadas'

In [8]:
app.layout = dbc.Container([
        dbc.Row([
            dbc.Col([
                html.H2(children='Vaccine Dashboard - Data Science Recruit',
                        className='text-center shadow-lg',
                       )
            ],width = {'size':10, 'order':1 }),
            
            dbc.Col([
                dbc.Card(
                    dbc.CardBody([
                        dbc.CardLink("Website Link", href="https://datasciencerecruit.com/"),
                    ])
                )
            ],width = {'size':2, 'order':2 }),
        ]),
            
    html.Br(),
    
    html.P("Please select the amount of days that you wish to make predictions. Check 0 if you just want to check the current vaccination situation"),
    
    dcc.RadioItems(id='days',
                   options=[{'label':0, 'value':0},
                            {'label':15, 'value':15},
                            {'label':30, 'value':30},
                            {'label':45, 'value':45},
                            {'label':60, 'value':60},
                            {'label':90, 'value':90},
                           ],value=30),
    
    dcc.Dropdown(id='country',
                 options=[{'label':i, 'value':j} for i,j in zip(list(locations_labels), locations_values)],
                 multi=False,
                 value='Brazil'),
            
    dcc.Graph(id='graph', figure={})
    
])


In [9]:
# Connecta Plotly graphs com Dash Components
@app.callback(
    Output(component_id='graph', component_property='figure'),
    Input(component_id='country', component_property='value'),
    Input(component_id='days', component_property='value')
)

def update_graph(country, days):
    file_path = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/country_data/{}.csv'.format(country)
    data = pd.read_csv(file_path, usecols=['date', 'total_vaccinations'])
    
    data['date'] = pd.to_datetime(data['date'])
    data.rename(columns={'date':'ds','total_vaccinations':'y'}, inplace=True)
    
    model = Prophet(yearly_seasonality=False,weekly_seasonality=True, daily_seasonality=True, seasonality_mode='additive')
    model.fit(data)
    future = model.make_future_dataframe(periods=days)
    predictions = model.predict(future)
    predictions.drop(data.index, inplace=True)

    trace = [go.Scatter(x=data['ds'], y=data['y']),
             go.Scatter(x=predictions['ds'], y=predictions['yhat'])]
    layout = go.Layout(title='Number of Vaccinated People - {}'.format(country.replace('%20', ' ')))
    fig = go.Figure(data=trace, layout=layout)
    
    return fig

In [10]:
if __name__ == '__main__':
    app.run_server(debug=True, port=8080, use_reloader=False)

Dash is running on http://127.0.0.1:8080/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


INFO:fbprophet:n_changepoints greater than number of observations. Using 0.
INFO:fbprophet:n_changepoints greater than number of observations. Using 0.
INFO:fbprophet:n_changepoints greater than number of observations. Using 0.
